In [ ]:
import sys, timm, torch
print(sys.version)
print(timm.__version__)
print(torch.__version__)

In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [Iterate Like a Grandmaster](https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster) I explained that when working on a Kaggle project:

> ...the focus generally should be two things:
> 
> 1. Creating an effective validation set
> 2. Iterating rapidly to find changes which improve results on the validation set.

Here I'm going to go further, showing the process I used to tackle the [Paddy Doctor](https://www.kaggle.com/competitions/paddy-disease-classification) competition, leading to four submissions in a row which all were (at the time of submission) in 1st place, each one more accurate than the last. You might be surprised to discover that the process of doing this was nearly entirely mechanistic and didn't involve any consideration of the actual data or evaluation details at all.

This notebook is the first in a series showing every step of the process. At the end of this notebook we'll have a basic submission; by the end of the series you'll see how I got to the top of the table!:

<img src="https://user-images.githubusercontent.com/346999/174389920-60d67ead-0f36-41d0-9649-e23b08720c8a.png" width="600"/>

As a special extra, I'm also opening up early a selection of "walkthru" videos that we've been preparing for the new upcoming fast.ai course. Each day I do a walkthru with fast.ai fellows and registered students, and we record those sessions. They'll all be released at the same time as the next course (probably August 2022), but I'm releasing the ones covering this competition right now! Here they are:

- [Walkthru 8](https://www.youtube.com/watch?v=-Scs4gbwWXg)
- [Walkthru 9](https://www.youtube.com/watch?v=EK5wJRzffas)
- [Walkthru 10](https://youtu.be/zhBRynq9Yvo)
- [Walkthru 11](https://youtu.be/j-zMF2VirA8)
- [Walkthru 12](https://youtu.be/GuCkpjXHdTc)
- [Walkthru 13](https://youtu.be/INrkhUGCXHg)

When you're done with this notebook, take a look at [part 2 of the series](https://www.kaggle.com/code/jhoward/small-models-road-to-the-top-part-2/).

## Getting set up

First, we'll get the data. I've just created a new library called [fastkaggle](https://fastai.github.io/fastkaggle/) which has a few handy features, including getting the data for a competition correctly regardless of whether we're running on Kaggle or elsewhere. Note you'll need to first accept the competition rules and join the competition, and you'll need your kaggle API key file `kaggle.json` downloaded if you're running this somewhere other than on Kaggle. `setup_comp` is the function we use in `fastkaggle` to grab the data, and install or upgrade our needed python modules when we're running on Kaggle:

In [ ]:
comp = 'paddy-disease-classification'

path = setup_comp(comp) #, install='fastai "timm>=0.6.2.dev0"')

In [ ]:
path

Now we can import the stuff we'll need from fastai, set a seed (for reproducibility -- just for the purposes of making this notebook easier to write; I don't recommend doing that in your own analysis however) and check what's in the data:

In [ ]:
from fastai.vision.all import *
# set_seed(42)

path.ls()

## Looking at the data

The images are in `train_images`, there're 10 folders under `train_images` with the folder name as the name of the disease, so let's grab a list of all of them:

In [ ]:
trn_path = path/'train_images'

In [ ]:
trn_path.ls()

In [ ]:
train_files = get_image_files(trn_path)

In [ ]:
type(train_files)

...and take a look at one:

In [ ]:
img = PILImage.create(train_files[20])
print(img.size)
img.to_thumb(128)

In [ ]:
type(img)

Looks like the images might be 480x640 -- let's check all their sizes. This is faster if we do it in parallel, so we'll use fastcore's `parallel` for this:

```
Function:
parallel(f, items, *args, n_workers=4, total=None, progress=None,
           pause=0, method=None, threadpool=False, timeout=None,
           chunksize=1, **kwargs)
```
* What this does: Applies func in parallel to items, using n_workers 


In [ ]:
from fastcore.parallel import *

def file_size(o): return PILImage.create(o).size
    
sizes = parallel(file_size, train_files, n_workers=10) # apply file_size function to all items under folder train_files, parallel to 10 workers
pd.Series(sizes).value_counts()

They're nearly all the same size, except for a few. Because of those few, however, we'll need to make sure we always resize each image to common dimensions first, otherwise fastai won't be able to create batches. For now, we'll just squish them to 480x480 images, and then once they're in batches we do a random resized crop down to a smaller size, along with the other default fastai augmentations provided by `aug_transforms`. We'll start out with small resized images, since we want to be able to iterate quickly:

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, 
                                   valid_pct=0.2,  # dls contains the validation set, make it easy to calc train and validation loss at the same time
                                   item_tfms = Resize(480, method='squish'),  # resize every single item into 480x480
                                   batch_tfms = aug_transforms(size=128, min_scale=0.75)  # randomly resize images to 128, with minimum keep portion to be 75%
                                  )

In [ ]:
dls.show_batch(max_n=10)

## Our first model

Let's create a model. To pick an architecture, we should look at the options in [The best vision models for fine-tuning](https://www.kaggle.com/code/jhoward/the-best-vision-models-for-fine-tuning). I like the looks of `resnet26d`, which is the fastest resolution-independent model which gets into the top-15 lists there.

3-step:
1. feed the train/valid set `dls` to a vision_learner with vision model `resnet26d`
2. find the best learning rate for the vision_learner
3. use the learning rate to fine tune the vision_learner in step 1

Question: what does fine tune do with learning rate?

Thought process: 
1. vision_learner is a pre-trained Neural Net + paddy data, this vision_learner should be able to predict the test set with a loss denoted as `loss_1`
2. And then we find the learning_rate that can minimize this loss in the fastest way, let's name this `loss_ideal`.
3. Then we feed the learning_rate to vision_learner (a NN) that changes the paramaters of the NN in the way that it descrease the `loss_1` towards `loss_ideal`

In classification problem, the loss is cross-entropy. (CrossEntropyLossFlat)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
learn = vision_learner(dls, 'resnet26d', metrics=error_rate, path='.')

Let's try `fit_one_cycle` first:

In [ ]:
learn.fit_one_cycle(5)

![read output](https://github.com/anna4fun/learning_notes_practical_deep_learning_courses/blob/main/Kaggle_jupyterNB/paddy_disease_classification/how_to_read_output_of_fit_one_cycle.png)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

The original notebook has `learn.to_fp16()` and that would crash when running the learning rate finders. I am changing to `to_fp32()` and avoid the crash.

In [ ]:
learn.to_fp32()

Let's see what the learning rate finder shows:

y axis is the loss, x axis shows the corresponding learning rate. The minimum of loss correspond to learning_rate = 0.1, so I will use this minimum loss learning rate first, see if it causes overfitting.

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide, minimum))

`lr_find` generally recommends rather conservative learning rates, to ensure that your model will train successfully. I generally like to push it a bit higher if I can. Let's train a few epochs and see how it looks:

learn.fine_tune() 
```
Learner.fine_tune (epochs, base_lr=0.002, freeze_epochs=1, lr_mult=100,
                    pct_start=0.3, div=5.0, lr_max=None,
                    div_final=100000.0, wd=None, moms=None, cbs=None,
                    reset_opt=False, start_epoch=0)
Fine tune with Learner.freeze for freeze_epochs, then with Learner.unfreeze for epochs, using discriminative LR.
```

In [ ]:
learn.fine_tune(3, 0.0005)

My interpretation:
1. train loss, valid loss, error rate all decrease monotonically, not indicating overfitting
2. time to train each epoch on average is 46.5 seconds, which is not bad


Noted that the loss here is cross-entropy loss, which is used for optimizing a classification problem. Error rate is the reportin metrics.

Difference between `fit_one_cycle` and `fine_tune`:
They reach to the same ends.fine_tune is a particular combination of fit_one_cycle(s) + (un)freeze(s), that works well in a lot (if not most) situations.

discussion: https://forums.fast.ai/t/fine-tune-vs-fit-one-cycle/66029

chatgpt answer:
![difference](https://github.com/anna4fun/learning_notes_practical_deep_learning_courses/blob/main/Kaggle_jupyterNB/paddy_disease_classification/difference_between_fit_one_cycle_AND_fine_tune.png)

We're now ready to build our first submission. Let's take a look at the sample Kaggle provided to see what it needs to look like:

## Submitting to Kaggle

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

OK so we need a CSV containing all the test images, in alphabetical order, and the predicted label for each one. We can create the needed test set using fastai like so:

In [ ]:
tst_files = get_image_files(path/'test_images').sorted()
tst_dl = dls.test_dl(tst_files)

We can now get the probabilities of each class, and the index of the most likely class, from this test set (the 2nd thing returned by `get_preds` are the targets, which are blank for a test set, so we discard them):

In [ ]:
probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
idxs

These need to be mapped to the names of each of these diseases, these names are stored by fastai automatically in the `vocab`:

In [ ]:
dls.vocab

We can create an apply this mapping using pandas:

In [ ]:
mapping = dict(enumerate(dls.vocab))
results = pd.Series(idxs.numpy(), name="idxs").map(mapping) # in pandas, mapping a dictionary is super fast
results

Kaggle expects the submission as a CSV file, so let's save it, and check the first few lines:

In [ ]:
ss['label'] = results
ss.to_csv('submission.csv', index=False)
!head submission.csv

Let's submit this to kaggle. We can do it from the notebook if we're running on Kaggle, otherwise we can use the API:

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'initial rn26d 128px', comp)

Success! We successfully created a submission.

## Conclusion

Our initial submission is not very good (top 80% of teams) but it only took a minute to train. The important thing is that we have a good starting point to iterate from, and we can do rapid iterations. Every step from loading the data to creating the model to submitting to Kaggle is all automated and runs quickly.

Therefore, we can now try lots of things quickly and easily and use those experiments to improve our results. In the next notebook, we'll do exactly that! So if you're ready, take a look at [part 2 of the series](https://www.kaggle.com/code/jhoward/small-models-road-to-the-top-part-2/).

If you found this notebook useful, please remember to click the little up-arrow at the top to upvote it, since I like to know when people have found my work useful, and it helps others find it too. And if you have any questions or comments, please pop them below -- I read every comment I receive!

## Addendum

`fastkaggle` also provides a function that pushes a notebook to Kaggle Notebooks. I wrote this notebook on my own machine, and pushed it to Kaggle from there -- here's the command I used:

In [ ]:
if not iskaggle:
    push_notebook('jhoward', 'first-steps-road-to-the-top-part-1',
                  title='First Steps: Road to the Top, Part 1',
                  file='first-steps-road-to-the-top-part-1.ipynb',
                  competition=comp, private=False, gpu=True)